In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import missingno as msno
import seaborn as sns
import datetime
import pickle

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('../intros/DS/avion/Dataset+Projet+4/'):
for dirname, _, filenames in os.walk('/kaggle/input/retard-avion/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Fusion des datasets
Lecture de l'ensemble des fichiers pour créer le Dataframe complet avec les champs à conserver

## Liste des colonnes à conserver
* FL_DATE = Date du départ Concaténation du mois, jour, année
* MONTH  = Mois de l'année
* DAY_OF_MONTH = Jour du mois
* UNIQUE_CARRIER = Code du transporteur
* ORIGIN_AIRPORT_ID = Identifiant de l'aéroport d'origine
* ORIGIN_CITY_NAME =  Nom de l'aéroport d'origine
* ORIGIN_STATE_ABR = Etat d'origine
* DEST_AIRPORT_ID = Identifiant de l'aéroport de destination
* DEST_CITY_NAME = Nom de l'aéroport de destination
* DEST_STATE_ABR = Etat de destination
* CRS_DEP_TIME = Heure de départ thérorique
* DEP_TIME = Heure de départ réelle
* DEP_DELAY = Calcul de la différence entre le départ théorique et réel
* ARR_DELAY = Calcul de la différence entre l'arrivée théorique et réelle ==> Champs qui nous indique le retard ou non de l'avion
* CRS_ELAPSED_TIME = Durée du vol théorique
* CRS_ARR_TIME = Heure d'arrivée théorique
* DISTANCE = Distance du vol
* DISTANCE_GROUP = Distance du vol regroupé en catégorie ==> Finalement pas retenu, meilleur résultat avec la distance

In [ ]:
ChampsAConserver = {'FL_DATE','MONTH','DAY_OF_MONTH','UNIQUE_CARRIER','ORIGIN_STATE_ABR','DEST_STATE_ABR','CRS_DEP_TIME',
                  'DEP_TIME','DEP_DELAY','ARR_DELAY','CRS_ELAPSED_TIME','CRS_ARR_TIME','DISTANCE'}

# ChampsAConserver = {'FL_DATE','MONTH','DAY_OF_MONTH','UNIQUE_CARRIER','ORIGIN_STATE_ABR','DEST_STATE_ABR','CRS_DEP_TIME',
#                     'DEP_TIME','ARR_DELAY','CRS_ELAPSED_TIME','CRS_ARR_TIME','DISTANCE'}

full_df = None
#for dirname, _, filenames in os.walk('../intros/DS/avion/Dataset+Projet+4/'):
for dirname, _, filenames in os.walk('/kaggle/input/retard-avion/'):
    for filename in filenames:
        file_path = os.path.join(os.path.abspath(dirname), filename)
        df = pd.read_csv(file_path, sep = ',', encoding='UTF-8', usecols=ChampsAConserver)
        
        if full_df is None:
            full_df = df
        else:            
            full_df = pd.concat([full_df, df], axis=0)

In [ ]:
#Affichage de la matrice de remplissage des données
msno.matrix(full_df)

In [ ]:
#Lecture d'un seul Fichier
df = pd.read_csv('/kaggle/input/retard-avion/2016_01.csv', sep = ',', encoding='UTF-8')
#df = pd.read_csv('../intros/DS/avion/Dataset+Projet+4/2016_01.csv', sep = ',', encoding='UTF-8')

In [ ]:
#Affichage d'une courbe de remplissage des attributs
indAvg = full_df.isna().mean().sort_values()
plt.figure(figsize=(10,10))
plt.plot(range(indAvg.shape[0]),indAvg.values)
plt.plot([0,indAvg.shape[0]],[0.7,0.7])
plt.xlabel('Attributs')
plt.ylabel('Taux de Null')
plt.title('Qualités des indicateurs')
plt.show()

In [ ]:
#Affichage d'un histogramme surles premières colonnes 
X = full_df[full_df.columns[:12]].values
fig = plt.figure(figsize=(16, 12))
for feat_idx in range(X.shape[1]):
    ax = fig.add_subplot(3,4, (feat_idx+1))
    h = ax.hist(X[:, feat_idx], bins=50, color='steelblue', density=True, edgecolor='none')
    ax.set_title(full_df.columns[feat_idx], fontsize=14)


# Changement des types
* pour faciliter la suite de l'analyse et supprimer les enregistrements qui ne peuvent être convertis
* Tri par date pour permettre le découpage temporel lors de la cross validation
* Création d'une colonne avec le nombre de jour par rapprt aux vacances

In [ ]:
#Correction d'une erreur de date en entrée
full_df = full_df[full_df['FL_DATE'] != '5059']
#Conversion
full_df['FL_DATE'] = pd.to_datetime(full_df['FL_DATE'])
#Tri par date
full_df = full_df.sort_values('FL_DATE')

In [ ]:
#Conversion des champs numériques
# modification des types du dataset en float quand cela est nécessaire
num_col = ["DEP_TIME","DAY_OF_MONTH","MONTH","DEP_DELAY", "ARR_DELAY", "CRS_ELAPSED_TIME", "DISTANCE"]

for col in num_col:
    print(col)
    full_df[col] = pd.to_numeric(full_df[col], errors='coerce')
#Utilisation de l'Option errors = coerce pour remplacer les erreurs de conversions avec NA

#Conversion de l'heure de départ en string (pour récupération de l'heure)
full_df['CRS_DEP_TIME'] = full_df['CRS_DEP_TIME'].astype('str') 

#Conversion de l'heure d'arrivée en string (pour récupération de l'heure)
full_df['CRS_ARR_TIME'] = full_df['CRS_ARR_TIME'].astype('str') 

#Suppression des lignes avec des NA
full_df = full_df.dropna(axis=0, subset=['DEP_TIME'])
full_df = full_df.dropna(axis=0, subset=['DAY_OF_MONTH'])
full_df = full_df.dropna(axis=0, subset=['MONTH'])
full_df = full_df.dropna(axis=0, subset=['DEP_DELAY'])
full_df = full_df.dropna(axis=0, subset=['ARR_DELAY'])
full_df = full_df.dropna(axis=0, subset=['CRS_ELAPSED_TIME'])
full_df = full_df.dropna(axis=0, subset=['DISTANCE'])
full_df = full_df.dropna(axis=0, subset=['CRS_DEP_TIME'])
full_df = full_df.dropna(axis=0, subset=['CRS_ARR_TIME'])

# Analyse métier des retards

In [ ]:
#Affichage des retards par transporteur
sns.set_theme(style="darkgrid")
sns.set(rc={'figure.figsize':(25,12)})
ax = sns.countplot(x="UNIQUE_CARRIER", data=full_df.loc[full_df['ARR_DELAY']>0])

In [ ]:
#Affichage des retards par état d'origine
ax = sns.countplot(x="ORIGIN_STATE_ABR", data=full_df.loc[full_df['ARR_DELAY']>0])

In [ ]:
#Affichage des retards par état de destination
ax = sns.countplot(x="DEST_STATE_ABR", data=full_df.loc[full_df['ARR_DELAY']>0])

In [ ]:
fig = plt.figure()
fig.suptitle('Retard Arrivée/Retard au départ')
plt.xlabel('Retard Arrivée')
plt.ylabel('Retard Départ')
plt.scatter(full_df.loc[full_df['ARR_DELAY']>120,'ARR_DELAY'], full_df.loc[full_df['ARR_DELAY']>120,'DEP_DELAY'],color = 'coral')

In [ ]:
full_df.loc[full_df['ARR_DELAY']>120].head(10)

In [ ]:
#Affichage des retards par Distance_Group
#Incrémentation du nombre tous les 250km. la dernière catégorie est donc plus de 2500km 
ax = sns.countplot(x="DISTANCE_GROUP", data=full_df.loc[full_df['ARR_DELAY']>0])

# Nombre de jours d'écart avec des vacances
L'idée est de créer une dimension supplémentaire avec le nombre de jours qui nous sépare des vacances les plus proche

In [ ]:
USHolydays = [datetime.date(2016,1,1),datetime.date(2016,1,18),datetime.date(2016,2,15),datetime.date(2016,5,30),datetime.date(2016,7,4),datetime.date(2016,9,5),datetime.date(2016,10,10),datetime.date(2016,11,11),datetime.date(2016,11,24),datetime.date(2016,12,26),datetime.date(2016,12,31)]
def daystoholydays(day,month):
    return min([abs((x - datetime.date(2016, month, day)).days) for x in USHolydays])

#Ajout de la colonne sur le nombre de jours jusqu'au prochaine vacance
full_df['daysToHolydays'] = full_df.loc[:,['DAY_OF_MONTH','MONTH']].apply(lambda x: daystoholydays(x['DAY_OF_MONTH'],x['MONTH']), axis = 1)

# Utilisation des Mois, Heures, jours
> Transformation des informations cyclique à l'aide des COS et SIN

In [ ]:
#Transformation du mois
month_in_year = 12
full_df['sin_month'] = np.sin(2*np.pi*full_df.MONTH/month_in_year)
full_df['cos_month'] = np.cos(2*np.pi*full_df.MONTH/month_in_year)
#Suppression de la colonne initiale
full_df.drop('MONTH', axis=1, inplace=True)

#Transformation du jours du mois
day_in_month = 30
full_df['sin_day'] = np.sin(2*np.pi*full_df.DAY_OF_MONTH/day_in_month)
full_df['cos_day'] = np.cos(2*np.pi*full_df.DAY_OF_MONTH/day_in_month)
#Suppression de la colonne initiale
full_df.drop('DAY_OF_MONTH', axis=1, inplace=True)

#Transformation de l'heure de départ
hour_in_day = 24
full_df['sin_hour_dep'] = np.sin(2*np.pi*pd.to_numeric(full_df.CRS_DEP_TIME.str[:2],errors='coerce')/hour_in_day)
full_df['cos_hour_dep'] = np.cos(2*np.pi*pd.to_numeric(full_df.CRS_DEP_TIME.str[:2],errors='coerce')/hour_in_day)
#Suppression de la colonne initiale
full_df.drop('CRS_DEP_TIME', axis=1, inplace=True)

#Transformation de l'heure d'arrivée
hour_in_day = 24
full_df['sin_hour_arr'] = np.sin(2*np.pi*pd.to_numeric(full_df.CRS_ARR_TIME.str[:2],errors='coerce')/hour_in_day)
full_df['cos_hour_arr'] = np.cos(2*np.pi*pd.to_numeric(full_df.CRS_ARR_TIME.str[:2],errors='coerce')/hour_in_day)
#Suppression de la colonne initiale
full_df.drop('CRS_ARR_TIME', axis=1, inplace=True)

# Mean Target Encoding
Finalement je n'utilise pas le One Hot Encoding car la multiplication des variables posent un problème. Je vais utiliser le Mean Target Encoding
* Création de plusieurs DataframeGoupby afin d'obtenir les écarts types de chaque catégorie

In [ ]:
#Création du DF en group by par état de départ
final_df_by_ORIGIN_STATE_ABR = full_df.groupby('ORIGIN_STATE_ABR')
#Calcul de la moyenne par état de départ
std_by_state_origin = final_df_by_ORIGIN_STATE_ABR['ARR_DELAY'].std()
for x in std_by_state_origin.keys() :
    full_df.loc[full_df['ORIGIN_STATE_ABR'] == x,'meanByStateOrigin'] =  std_by_state_origin[x]
    
    
#Création du DF en group by par état d'arrivée
final_df_by_ORIGIN_STATE_ABR = full_df.groupby('DEST_STATE_ABR')
#Calcul de la moyenne par état d'arrivée
std_by_state_dest = final_df_by_ORIGIN_STATE_ABR["ARR_DELAY"].std()
for x in std_by_state_dest.keys() :
    full_df.loc[full_df['DEST_STATE_ABR'] == x,'meanByStateDest'] =  std_by_state_dest[x]
    
#Création du DF en group by par compagnie
final_df_by_CARRIER = full_df.groupby('UNIQUE_CARRIER')
#Calcul de la moyenne compagnie
std_by_carrier = final_df_by_CARRIER["ARR_DELAY"].std()
for x in std_by_carrier.keys() :
    full_df.loc[full_df['UNIQUE_CARRIER'] == x,'meanByCarrier'] =  std_by_carrier[x]

In [ ]:
#Création du DF en group by etat d'origine et de départ (pour le site WEB afin de ne pas saisir les données)
final_df_by_ORIGIN_STATE_ABR = full_df.groupby(['ORIGIN_STATE_ABR','DEST_STATE_ABR'])
#Calcul de la distance moyenne et du temps moyen entre deux états
mean_by_distance_state = final_df_by_ORIGIN_STATE_ABR["DISTANCE"].mean()
mean_by_time_state = final_df_by_ORIGIN_STATE_ABR["CRS_ELAPSED_TIME"].mean()

In [ ]:
#Création du DF en group by pour calculer le retard au départ en fonction de la compagnie, état de départ et état d'arrivée (Pour le site WEB)
final_df_by_CARRIER_STATE = full_df.groupby(['UNIQUE_CARRIER','ORIGIN_STATE_ABR','DEST_STATE_ABR'])
mean_by_delay = final_df_by_CARRIER_STATE["ARR_DELAY"].mean()

# Sérialisation des Dataframes
Pour la prédiction, j'ai besoin des éléments suivants qui ne seront pas saisis par l'utlisateur
* L'écart type des retards pour l'état d'origine (Mean target encoding)
* L'écart type des retards pour l'état d'arrivée (Mean target encoding)
* l'écart type des retards d'une compagnie (Mean target encoding)
* La distance moyenne entre deux états
* Le temps moyen de vol entre deux états
* le retard moyen d'une compagnie entre deux états (prédiction du retard au décollage)

In [ ]:
df_std_state_origin = std_by_state_origin.to_frame()
df_std_state_dest = std_by_state_dest.to_frame()
df_std_carrier = std_by_carrier.to_frame()
df_mean_distance = mean_by_distance_state.to_frame()
df_mean_time = mean_by_time_state.to_frame()
df_mean_by_delay = mean_by_delay.to_frame()
pickle.dump(df_std_state_origin, open('df_std_state_origin.pickle', 'wb')) #Saving DF
pickle.dump(df_std_state_dest, open('df_std_state_dest.pickle', 'wb')) #Saving DF
pickle.dump(df_std_carrier, open('df_std_carrier.pickle', 'wb')) #Saving DF
pickle.dump(df_mean_distance, open('df_mean_distance.pickle', 'wb')) #Saving DF
pickle.dump(df_mean_time, open('df_mean_time.pickle', 'wb')) #Saving DF
pickle.dump(df_mean_by_delay, open('df_mean_by_delay.pickle', 'wb')) #Saving DF

# Mise à plat des catégories
Création de colonnes boolean pour l'utlisation des dimensions catégorielles

Je ne l'utilise plus finalement car je fais du Mean Target Encoding sur les variables catégorielles

In [ ]:
#Mise à plate des données catégorielles
#final_df = pd.get_dummies(full_df, columns = {'ORIGIN_STATE_ABR','DEST_STATE_ABR','UNIQUE_CARRIER'})
#final_df = pd.get_dummies(full_df, columns = {'UNIQUE_CARRIER'})

In [ ]:
#Suppression des colonnes inutiles
full_df.drop({'FL_DATE','DEP_TIME','ORIGIN_STATE_ABR','DEST_STATE_ABR','UNIQUE_CARRIER'}, axis=1, inplace=True)

#full_df.drop({'FL_DATE','DEP_DELAY','DEP_TIME','ORIGIN_STATE_ABR','DEST_STATE_ABR','UNIQUE_CARRIER'}, axis=1, inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)
full_df.head(10)

# Création de deux jeux de données
* final_df va contenir l'ensemble des données et des dimensions
* df_for_reg_lineaire pour tenter d'améliorer la régresssion linéaire sans les outlers sur le ARR_DELAY >120 ou <-20 ==> Plus utlisé finalement car n'a rien apporté

In [ ]:
#Creation d'un Dataset simple sans les outlers pour la régression linéaire
full_df = full_df.loc[(full_df['ARR_DELAY']<=120) & (full_df['ARR_DELAY']>-20)]
# df_for_reg_lineaire = df_for_reg_lineaire.iloc[:,0:12]
#df_for_reg_lineaire = final_df.iloc[:,0:15]

# Ecriture du fichier de sortie pour l'utiliser dans Modèle

In [ ]:
#Ecriture du fichier de sortie pour l'utiliser dans le programme modèle
final_df.to_csv('retardAvionClean.csv', sep = ',', encoding='UTF-8')

# Partie Modèle (à reporter dans l'autre Notebook à la fin)

# Création de deux échantillons avec 10% des données
* sub_df
* suf_df_for_reg_lineaire => Plus utilisé finalement avec le mean target encoding

In [ ]:
ind = np.random.choice(full_df.shape[0], round(full_df.shape[0]*0.1))
sub_df = full_df.iloc[ind]
#ind = np.random.choice(df_for_reg_lineaire.shape[0], round(df_for_reg_lineaire.shape[0]*0.1))
#suf_df_for_reg_lineaire = df_for_reg_lineaire.iloc[ind]

In [ ]:
#suppression des Dataframes initiaux
del df_for_reg_lineaire
del final_df

# Création du jeu d'entrainement et de test
Utlisation de train_test_split qui va découper les données avec le pourcentage spécifié

In [ ]:
from sklearn.model_selection import train_test_split
# On récupère les features d'un côté...
X = sub_df.drop('ARR_DELAY', axis=1).to_numpy()
#X_forReg = suf_df_for_reg_lineaire.drop('ARR_DELAY', axis=1).to_numpy()

# et les labels de l'autre
y = sub_df[['ARR_DELAY']].to_numpy()
#y_forReg = suf_df_for_reg_lineaire[['ARR_DELAY']].to_numpy()

#Création d'un jeu d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#X_train_forReg, X_test_forReg, y_train_forReg, y_test_forReg = train_test_split(X_forReg, y_forReg, test_size=0.33)

#Suppression des variables temporaires
del X,y

# Standardisation
Normalisation et réduction du jeu d'entrainement et du jeu de test

In [ ]:
#Création des jeux de données standardisés
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)

#std_scale = preprocessing.StandardScaler().fit(X_test)
X_test_std = std_scale.transform(X_test)

#Pour la régression linéaire
#std_scale = preprocessing.StandardScaler().fit(X_train_forReg)
#X_train_forReg_std = std_scale.transform(X_train_forReg)

#std_scale = preprocessing.StandardScaler().fit(X_test_forReg)
#X_test_forReg_std = std_scale.transform(X_test_forReg)

# Création de différents modèles
## Baseline
Création d'un modèle très basique pour avoir une référence basse

* Utlisation du modèle DummyRegressort qui ressort la moyenne

In [ ]:
#Import pour la partie modèles
import sklearn.dummy
import sklearn.metrics
import sklearn.model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
import sklearn.linear_model
import sklearn.tree
import sklearn.random_projection
from sklearn.neighbors import KNeighborsRegressor

#Création des objetfs transverses
tscv = TimeSeriesSplit()
resultat = {}

# Baseline
Création d'un modèle très basique pour avoir une référence basse

* Utlisation du modèle DummyRegressort qui ressort la moyenne

In [ ]:
model = sklearn.dummy.DummyRegressor()
model.fit(X_train_std, y_train)
preds = model.predict(X_test_std)
print(sklearn.metrics.r2_score(y_test, preds))
print(sklearn.metrics.mean_absolute_error(y_test, preds))

In [ ]:
#CROSS VALIDATION 5FOLDS
model = sklearn.dummy.DummyRegressor()
sklearn.model_selection.cross_val_score(model, X_train_std, y_train, cv=tscv.split(X_train_std, y_train))

# Indicateurs Regression

* explained_variance = metrics.explained_variance_score
* max_error = metrics.max_error
* neg_mean_absolute_error = metrics.mean_absolute_error
* neg_mean_squared_error = metrics.mean_squared_error
* neg_root_mean_squared_error = metrics.mean_squared_error
* neg_mean_squared_log_error = metrics.mean_squared_log_error
* neg_median_absolute_error = metrics.median_absolute_error
* R2 = metrics.r2_score
* neg_mean_poisson_deviance = metrics.mean_poisson_deviance
* neg_mean_gamma_deviance = metrics.mean_gamma_deviance
* neg_mean_absolute_percentage_error = metrics.mean_absolute_percentage_error

In [ ]:
#Fonction qui consolide les différents modèles dans un tableau avec leurs résultats
def consolidationResultat_model(resultat, model, X_test, y_reel , nomModel):
    y_pred = model.predict(X_test)
    r2 = sklearn.metrics.r2_score(y_reel, y_pred)
    mae = sklearn.metrics.mean_absolute_error(y_reel, y_pred)
    SqrError = sklearn.metrics.mean_squared_error(y_reel, y_pred)
    resultat[nomModel] = {r2,mae,SqrError}
    return resultat

In [ ]:
#GRID SEARCH pour trouver les HYPERPARAMETRES ==> PAS DANS CET EXEMPLE
params = {}
model = sklearn.dummy.DummyRegressor()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test,'DummyRegressor')

In [ ]:
#GRID SEARCH sur régression linéaire ==> PAS DE PARAMETRE DANS CET EXEMPLE
params = {}
model = sklearn.linear_model.LinearRegression()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'Régression Linéaire')

#Vérifier si une obeservation est complètement aberrante

In [ ]:
model = sklearn.linear_model.LinearRegression()
model.fit(X_train_std, y_train)
y_pred  = model.predict(X_test_std)

print(sklearn.metrics.mean_absolute_error(y_test, y_pred))
print(sklearn.metrics.r2_score(y_test, y_pred))

In [ ]:
fig = plt.figure(1, figsize=(6, 8))
plt.barh(full_df.loc[:, full_df.columns != 'ARR_DELAY'].columns.values,model.coef_[0],)

In [ ]:
plt.barh(full_df.loc[:, full_df.columns != 'ARR_DELAY'].columns.values,model.feature_importances_)

In [ ]:
fig = plt.figure()
fig.suptitle('Prediction/Réel')
plt.xlabel('Valeur Réelle')
plt.ylabel('Prédiction')
plt.scatter(y_test, y_pred,color = 'coral', alpha=0.1)

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(y_train_forReg)
plt.ylim(-100,250)
plt.show()

In [ ]:
#GRID SEARCH avec trouver l'Alpha de la régularisation Ridge
params = {'alpha':[1, 10]}
model = sklearn.linear_model.Ridge()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'Régression Ridge')

# Avantages de l’arbre de régression par rapport au modèle de régression linéaire sont les suivants :
* Il est plus simple et plus direct dans son approche
* La structure liant Y à X1, . . . , Xp n’importe pas ; celle-ci peut être linéaire ou autre
* Il n’y a pas d’hypothèse mathématique sous-jacente (pas d’hypothèse de normalité ou autre)
* Les dépendances éventuelles entre X1, . . . , Xp ne posent pas de problème

In [ ]:
#GRID SEARCH sur arbre de décision
params = {'max_depth': range(8, 9, 1)}
model = sklearn.tree.DecisionTreeRegressor()
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'Arbre de décision')

#MAX_DEPTH à 8 semble être le meilleur hyperparamètre
#Utiliser les variables les plus importantes features importance

# Sérialisation du modèle pour le site Web

In [ ]:
#Serialisation du modèle de regression linéaire
pickle.dump(model, open('RegressionLineaire.pickle', 'wb')) #Saving the model
pickle.dump(std_scale, open('scaler.pickle', 'wb')) #Saving the scaler

In [ ]:
#Serialisation du modèle d'arbre de décision
pickle.dump(model, open('arbreDecision.pickle', 'wb')) #Saving the model

In [ ]:
model = sklearn.tree.DecisionTreeRegressor(max_depth = 8)
model.fit(X_train_std, y_train)

y_pred  = model.predict(X_test_std)

fig = plt.figure()
fig.suptitle('Prediction/Réel')
plt.xlabel('Valeur Réelle')
plt.ylabel('Prédiction')
plt.scatter(y_test, y_pred,color = 'coral', alpha=0.1)

In [ ]:
ValAbsoErreur = [abs(y_test_elt - y_pred_elt) for y_test_elt, y_pred_elt in zip(y_test , y_pred)]
fig = plt.figure()
fig.suptitle('Erreur/Valeur attendue')
plt.xlabel('Valeur Attendue')
plt.ylabel('Valeur absolue de l''Erreur')
plt.scatter(y_test, ValAbsoErreur,color = 'coral', alpha=0.1)

In [ ]:
jeuDonnee = np.c_[y_test, ValAbsoErreur]
jeuDonnee = np.c_[jeuDonnee, ValAbsoErreur]
sns.scatterplot(x = jeuDonnee[:,0], y = jeuDonnee[:,1], data = jeuDonnee)
# print(type(y_test))
# print(type(ValAbsoErreur))


In [ ]:
#GRID SEARCH sur Random Forest
from sklearn.ensemble import RandomForestRegressor
params = {'max_depth': range(8, 14, 2)}
model = sklearn.ensemble.RandomForestRegressor(n_estimators = 30)
clf = GridSearchCV(model, params, cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test, 'RandomForest')

#max_depth=10 pour la valeur optimale. Refaire avec n_estimators plus grand

In [ ]:
#GRID SEARCH sur LGBM
import lightgbm as lgb
params = {
    'max_depth': range(13, 16,2)
}
model = lgb.LGBMRegressor()
clf = GridSearchCV(model, params,  cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test,'LGBM')
#max_depth=15 pour la valeur optimale.

In [ ]:
#GRID SEARCH KNN Regressor
params = {
    'n_neighbors': range(5, 12,1)
}
model = sklearn.neighbors.KNeighborsRegressor()
clf = GridSearchCV(model, params,  cv=tscv.split(X_train_std, y_train))
clf.fit(X_train_std, y_train)
consolidationResultat_model(resultat, clf.best_estimator_, X_test_std, y_test,'KNN Regressor')

#n_neighbors=10 valeur optimale

In [ ]:
model = sklearn.neighbors.KNeighborsRegressor(n_neighbors = 10)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
r2 = sklearn.metrics.r2_score(y_test, y_pred)